In [1]:
%matplotlib inline
# alternatively: widget, if installed and working, gives nicer inline plots
# %pylab widget
# fix failing variable completion:
# %config Completer.use_jedi = False

# First, Simplest Usecase

Let's optimize a simple ellipsoid function, $\sum_{i=1}^n 1000^{\frac{i-1}{n-1}}x_i^2$, whose Hessian has a condition number of $10^6$:

In [2]:
import cma

fun = cma.ff.elli  # we could use `functools.partial(cma.ff.elli, cond=1e4)` to change the condition number to 1e4
x0 = 4 * [2]  # initial solution
sigma0 = 1    # initial standard deviation to sample new solutions

x, es = cma.fmin2(fun, x0, sigma0)

(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=916991, Mon Dec  6 16:20:18 2021)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      8 1.627558087278906e+05 1.0e+00 1.02e+00  1e+00  1e+00 0:00.0
    2     16 1.113885569713139e+05 1.3e+00 9.68e-01  9e-01  1e+00 0:00.0
    3     24 1.236023557505223e+05 1.3e+00 8.86e-01  7e-01  1e+00 0:00.0
  100    800 2.047733508270783e-02 5.6e+02 7.35e-02  2e-04  9e-02 0:00.1
  200   1600 7.364161834974983e-15 8.2e+02 2.33e-06  6e-11  5e-08 0:00.2
termination on tolfun=1e-11 (Mon Dec  6 16:20:19 2021)
final/bestever f-value = 2.191368e-15 2.191368e-15
incumbent solution: [1.2162853398733964e-08, 3.5042136503204363e-09, -2.1603579683067802e-10, 1.8675297805320814e-11]
std deviation: [5.036667477332108e-08, 5.824321564766107e-09, 6.312816057683408e-10, 6.473395280007542e-11]


The return values are
- `x`, the best evaluted solution and
- `es`, the `cma.CMAEvolutionStrategy` class instance used to run the optimization.

The latter contains all available information about the run and the more complete return value under `es.result`.

In [3]:
es.result

CMAEvolutionStrategyResult(xbest=array([ 1.21628534e-08,  3.50421365e-09, -2.16035797e-10,  1.86752978e-11]), fbest=2.1913677367483816e-15, evals_best=1601, evaluations=1601, iterations=200, xfavorite=array([ 1.21628534e-08,  3.50421365e-09, -2.16035797e-10,  1.86752978e-11]), stds=array([5.03666748e-08, 5.82432156e-09, 6.31281606e-10, 6.47339528e-11]), stop={'tolfun': 1e-11})

In [4]:
es.result_pretty()  # shows some pretty information and returns es.result

termination on tolfun=1e-11
final/bestever f-value = 2.191368e-15 2.191368e-15
incumbent solution: [1.2162853398733964e-08, 3.5042136503204363e-09, -2.1603579683067802e-10, 1.8675297805320814e-11]
std deviation: [5.036667477332108e-08, 5.824321564766107e-09, 6.312816057683408e-10, 6.473395280007542e-11]


CMAEvolutionStrategyResult(xbest=array([ 1.21628534e-08,  3.50421365e-09, -2.16035797e-10,  1.86752978e-11]), fbest=2.1913677367483816e-15, evals_best=1601, evaluations=1601, iterations=200, xfavorite=array([ 1.21628534e-08,  3.50421365e-09, -2.16035797e-10,  1.86752978e-11]), stds=array([5.03666748e-08, 5.82432156e-09, 6.31281606e-10, 6.47339528e-11]), stop={'tolfun': 1e-11})

# Options and Plotting
Let's plot some data from the optimization. The above call to `cma.fmin2` has (by default) written data to the default output folder `outcmaes`. These data will be overwritten each time the function is called again. To prevent this, we could set/change the output folder with the `verb_filenameprefix` option. A effective way to find out this kind of behavior is to query the CMA options with a search string and read the comments in the return "value".

In [5]:
cma.CMAOptions('file')

{'signals_filename': 'cma_signals.in  # read versatile options from this file (use `None` or `""` for no file) which contains a single options dict, e.g. ``{"timeout": 0}`` to stop, string-values are evaluated, e.g. "np.inf" is valid',
 'verb_append': '0  # initial evaluation counter, if append, do not overwrite output files',
 'verb_filenameprefix': 'outcmaes/  # output path and filenames prefix',
 'verb_log': '1  #v verbosity: write data to files every verb_log iteration, writing can be time critical on fast to evaluate functions'}

Plotting is as simple as

In [6]:
%matplotlib widget
es.plot();  # cma.plot() does the same, as we just plot the data from the default output folder

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

A description of the output is given in the caption of Figure 3 in [Hansen et al (2015): Evolution Strategies](https://hal.inria.fr/hal-01155533/file/es-overview-2015.pdf#page=17).

# Options and Bound Constraints
We can add a boundary condition on any variable. We can find out how to do this, as mentioned above, by investigating the available boundary related options:

In [7]:
cma.CMAOptions('bound')  # returns options that contain "bound"

{'BoundaryHandler': 'BoundTransform  # or BoundPenalty, unused when ``bounds in (None, [None, None])``',
 'bounds': '[None, None]  # lower (=bounds[0]) and upper domain boundaries, each a scalar or a list/vector'}

Let's bound the second variable from below to values $\ge1$ using the `'bounds'` option. In this case, we must also take care that the initial solution is within the given bounds (it is, in our case). `'bounds'` is a list of two values, `[lower_bounds, upper_bounds]`, where `lower_bounds` and `upper_bounds` can be a scalar that applies to all variables or a list. If the list is shorter than the initial solution, the last value is applied to the remaining dimensions.

In [8]:
x, es = cma.fmin2(fun, x0, sigma0, {'bounds': [[None, 1, None], None]})

(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=843583, Mon Dec  6 16:20:21 2021)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      8 3.350768421831981e+05 1.0e+00 1.01e+00  1e+00  1e+00 0:00.0
    2     16 2.008930978755744e+05 1.2e+00 1.11e+00  1e+00  1e+00 0:00.0
    3     24 8.328670663162784e+04 1.6e+00 1.15e+00  1e+00  1e+00 0:00.0
  100    800 1.000005270650480e+02 6.1e+02 1.06e-02  2e-05  1e-02 0:00.1
  183   1464 9.999999999999997e+01 1.2e+03 1.18e-06  5e-11  5e-08 0:00.2
termination on tolfun=1e-11 (Mon Dec  6 16:20:21 2021)
final/bestever f-value = 1.000000e+02 1.000000e+02
incumbent solution: [-2.8111802702419823e-08, 1.0, 2.7004925912948e-10, -3.440019505637813e-11]
std deviation: [5.313228101255525e-08, 2.237375396275545e-09, 4.847367062276775e-10, 4.676257410886614e-11]


In [11]:
%matplotlib widget
es.plot();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
%matplotlib widget
es.plot(xsemilog=True, x_opt=[0, 1] + (len(x0) - 2) * [0]);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Other [`notebooks and recipes`](https://github.com/CMA-ES/pycma/notebooks), as this one, can be found at [https://github.com/CMA-ES/pycma/notebooks](https://github.com/CMA-ES/pycma/notebooks)

More `notebook-recipes` to come...